# 03 - The Learning Path Test Notebook

## Welcome to Your Assessment

Congratulations on completing the Claude Agent SDK tutorial series! This notebook is designed to test your understanding of the core concepts you've learned across all three notebooks. Instead of passive reading, you'll actively implement solutions to real-world scenarios.

---

## The Backstory: DataFlow Labs

You've just been hired as the **first AI/Agent Engineering hire** at **DataFlow Labs**, a growing data analytics company with 80 employees. The CEO, Alex Rivera, has a vision: build an internal AI agent ecosystem that helps different departments work more efficiently.

Your mission over the next few weeks:

1. **Week 1**: Build a prototype research agent for the Data Science team
2. **Week 2**: Expand it with enterprise features for the Operations team
3. **Week 3**: Integrate external systems for the DevOps team

Each week builds on the previous one, and at the end, you'll have a production-ready agent system.

---

## How This Test Works

Each exercise presents a scenario and code with **missing parts marked by `# TODO: ...` comments**. Your task is to:

1. Read the scenario and requirements
2. Fill in the missing code based on what you learned
3. Run the verification cell to check your solution

**Scoring Guide:**
- Each exercise is worth points based on complexity
- Partial credit for partially correct solutions
- Total possible: **100 points**

**Hints:**
- Review the original notebooks if you get stuck
- Pay attention to parameter names and types
- Remember the patterns you've seen

Let's begin!

---

## Setup

Run this cell to install dependencies and configure the environment:

In [ ]:
%%capture
%pip install -U claude-agent-sdk python-dotenv

In [ ]:
import json
import os
from dotenv import load_dotenv

from claude_agent_sdk import ClaudeAgentOptions, ClaudeSDKClient, query
from utils.agent_visualizer import print_activity, print_final_result, visualize_conversation

load_dotenv()

# Test your setup
print("✅ Setup complete! You're ready to begin.")

---

# Phase 1: The Prototype (Week 1)

## DataFlow Labs - Data Science Team Request

**From:** Dr. Sarah Chen, Head of Data Science  
**To:** You (AI/Agent Engineer)  
**Subject:** Research Assistant Prototype

> *"Our team spends 3-4 hours daily researching new ML techniques, datasets, and industry trends. We need a simple research agent that can search the web and synthesize findings. Start with a basic prototype - we can add features later."*

This phase tests your understanding of **Notebook 00: Research Agent** concepts.

---

### Exercise 1.1: Your First Stateless Query (10 points)

**Scenario:** Dr. Chen wants to test the concept with a simple query about recent machine learning trends.

**Your Task:** Complete the `ClaudeAgentOptions` configuration to:
1. Use the `claude-sonnet-4-5` model
2. Allow only the `WebSearch` tool

**Hint:** Remember the `query()` function creates stateless interactions.

In [ ]:
# Exercise 1.1: Complete the options configuration

messages = []

async for msg in query(
    prompt="What are the top 3 machine learning trends in 2025? Be concise.",
    options=ClaudeAgentOptions(
        # TODO: Set the model to claude-sonnet-4-5
        model="",  # Fill this in
        
        # TODO: Allow only the WebSearch tool
        allowed_tools=[],  # Fill this in
    ),
):
    print_activity(msg)
    messages.append(msg)

print_final_result(messages)

**Verification:** Run the cell above. If successful, you should see:
- `🤖 Thinking...`
- `🤖 Using: WebSearch()`
- A summarized response about ML trends
- Cost and duration information

---

### Exercise 1.2: Stateful Conversations with ClaudeSDKClient (15 points)

**Scenario:** Dr. Chen is impressed! Now she wants to have a multi-turn conversation where the agent remembers previous context. She'll ask about a dataset, then follow up with questions about it.

**Your Task:** Complete the code to:
1. Configure `ClaudeAgentOptions` with the model, system prompt, and allowed tools (WebSearch and Read)
2. Use `ClaudeSDKClient` as an async context manager
3. Send the first query and collect responses
4. Send a follow-up query that builds on the first response

**Hint:** The `async with ClaudeSDKClient()` pattern maintains conversation state across multiple `query()` calls.

In [ ]:
# Exercise 1.2: Complete the stateful conversation

messages = []

# TODO: Complete the ClaudeSDKClient setup
async with ClaudeSDKClient(
    options=ClaudeAgentOptions(
        # TODO: Set the model
        model="",  # Fill this in
        
        # TODO: Add a system prompt for a data science research specialist
        system_prompt="",  # Fill this in
        
        # TODO: Allow WebSearch and Read tools
        allowed_tools=[],  # Fill this in
    )
) as agent:
    # First query
    print("📝 Query 1: Asking about ImageNet dataset...\n")
    
    # TODO: Send the first query using agent.query()
    await agent.query("")  # Fill in the prompt about ImageNet
    
    # TODO: Collect responses using agent.receive_response()
    async for msg in agent.receive_response():
        print_activity(msg)
        messages.append(msg)
    
    print("\n" + "="*50 + "\n")
    print("📝 Query 2: Follow-up question (agent should remember context)...\n")
    
    # Second query - this should use context from the first
    # TODO: Send a follow-up query that references the first response
    await agent.query("")  # Fill in a follow-up prompt like "What are some alternatives to it?"
    
    async for msg in agent.receive_response():
        print_activity(msg)
        messages.append(msg)

print_final_result(messages)

**Verification:** The second query should demonstrate that the agent remembers the context from the first query (e.g., it knows you're talking about ImageNet without you repeating it).

---

### Exercise 1.3: Custom Activity Handler (10 points)

**Scenario:** Dr. Chen wants more detailed logging to understand what the agent is doing. She asks you to create a custom activity handler that shows more information.

**Your Task:** Complete the `custom_activity_handler` function to:
1. Check if the message is from the Assistant (class name contains "Assistant")
2. If it has content with a `name` attribute, print the tool name
3. Otherwise print "Thinking..."
4. Handle User messages (tool completions)

**Hint:** Look at the `get_activity_text` function pattern from the agent implementations.

In [ ]:
# Exercise 1.3: Implement a custom activity handler

def custom_activity_handler(msg):
    """Custom activity handler with detailed logging"""
    try:
        # TODO: Check if this is an Assistant message
        if "" in msg.__class__.__name__:  # Fill in the class name to check
            # Check if content exists and has items
            if hasattr(msg, "content") and msg.content:
                first_content = msg.content[0] if isinstance(msg.content, list) else msg.content
                
                # TODO: Check if this is a tool use (has 'name' attribute)
                if hasattr(first_content, ""):  # Fill in the attribute to check
                    print(f"🔧 [TOOL CALL] {first_content.name}")
                    return
            
            print("💭 [THINKING] Processing...")
        
        # TODO: Check if this is a User message (tool result)
        elif "" in msg.__class__.__name__:  # Fill in the class name to check
            print("✅ [TOOL RESULT] Received")
            
    except (AttributeError, IndexError):
        pass


# Test your handler
messages = []
async for msg in query(
    prompt="What is the capital of France? Use one websearch.",
    options=ClaudeAgentOptions(
        model="claude-sonnet-4-5",
        allowed_tools=["WebSearch"],
    ),
):
    custom_activity_handler(msg)
    messages.append(msg)

print(f"\n📊 Total messages processed: {len(messages)}")

**Verification:** You should see output like:
```
💭 [THINKING] Processing...
🔧 [TOOL CALL] WebSearch
✅ [TOOL RESULT] Received
💭 [THINKING] Processing...
```

---

## Phase 1 Complete! 🎉

**Points Possible: 35**

You've demonstrated understanding of:
- ✅ Stateless queries with `query()`
- ✅ Stateful conversations with `ClaudeSDKClient`
- ✅ Tool permissions with `allowed_tools`
- ✅ System prompts for specialization
- ✅ Activity handler patterns

Dr. Chen is happy with the prototype. Now it's time for Week 2!

---

# Phase 2: Enterprise Features (Week 2)

## DataFlow Labs - Operations Team Request

**From:** Marcus Thompson, VP of Operations  
**To:** You (AI/Agent Engineer)  
**Subject:** Enterprise-Grade Agent System

> *"The Data Science team's prototype is impressive! Our Operations team needs something more sophisticated. We need agents that can delegate to specialists, adapt their output style for different audiences, and maintain audit trails. Think of it as a Chief of Staff that can coordinate different functions."*

This phase tests your understanding of **Notebook 01: Chief of Staff Agent** concepts.

---

### Exercise 2.1: Configuring Output Styles (10 points)

**Scenario:** Marcus wants the agent to communicate differently with executives vs. engineers. You need to configure the agent to use custom output styles.

**Your Task:** Complete the `settings` parameter to specify an output style.

**Hint:** Output styles are passed as a JSON string in the `settings` parameter with the key `outputStyle`.

In [ ]:
# Exercise 2.1: Configure output styles

# First, let's create a simple output style test
# We'll use the chief_of_staff_agent directory which has output styles defined

messages = []

async with ClaudeSDKClient(
    options=ClaudeAgentOptions(
        model="claude-sonnet-4-5",
        cwd="chief_of_staff_agent",  # This directory has output styles defined
        
        # TODO: Configure the settings to use the "executive" output style
        # The settings parameter takes a JSON string
        settings='',  # Fill this in - should be a JSON string like '{"key": "value"}'
    )
) as agent:
    await agent.query("Describe your communication style in 2 sentences.")
    async for msg in agent.receive_response():
        print_activity(msg)
        messages.append(msg)

# Print the result
if messages and hasattr(messages[-1], 'result'):
    print(f"\n📋 Response:\n{messages[-1].result}")

**Verification:** The response should reflect executive-style communication: concise, metrics-focused, with clear recommendations.

---

### Exercise 2.2: Understanding Subagent Definitions (15 points)

**Scenario:** Marcus wants specialized subagents for different functions. You need to understand the markdown format for defining subagents.

**Your Task:** Write the frontmatter and beginning of a subagent definition for a "data-analyst" subagent.

**Requirements:**
1. Name: `data-analyst`
2. Description: Explain what this agent does (data analysis, visualization, insights)
3. Tools: Allow `Read`, `Bash`, and `WebSearch`
4. Start the system prompt section

**Hint:** Look at the format used in `.claude/agents/financial-analyst.md`

In [ ]:
# Exercise 2.2: Write a subagent definition

# Complete this markdown content for a data-analyst subagent
subagent_definition = """
---
name: 
description: 
tools: 
---

You are a senior data analyst for DataFlow Labs...

## Your Responsibilities

1. **Data Analysis**
   - Analyze datasets and extract insights
   - Create statistical summaries
   - Identify trends and patterns
"""

# Validation
print("Your subagent definition:")
print("=" * 50)
print(subagent_definition)
print("=" * 50)

# Check if the frontmatter is correctly formatted
import re
frontmatter_match = re.search(r'---\s*\nname:\s*(.+)\ndescription:\s*(.+)\ntools:\s*(.+)\n---', subagent_definition)

if frontmatter_match:
    name, desc, tools = frontmatter_match.groups()
    print("\n✅ Frontmatter parsed successfully!")
    print(f"   Name: {name.strip()}")
    print(f"   Description: {desc.strip()[:50]}...")
    print(f"   Tools: {tools.strip()}")
    
    # Check specific requirements
    if "data-analyst" in name.lower():
        print("   ✅ Name is correct")
    else:
        print("   ❌ Name should be 'data-analyst'")
        
    if all(tool in tools for tool in ["Read", "Bash", "WebSearch"]):
        print("   ✅ All required tools included")
    else:
        print("   ❌ Missing some tools (need Read, Bash, WebSearch)")
else:
    print("\n❌ Frontmatter format incorrect. Check the YAML structure.")

---

### Exercise 2.3: Permission Modes (10 points)

**Scenario:** Before executing major decisions, Marcus wants the agent to create a plan for review. You need to configure the agent in "plan" mode.

**Your Task:** Complete the options to:
1. Set the permission mode to `plan`
2. Configure allowed tools appropriately

**Hint:** The `permission_mode` parameter controls whether the agent executes or just plans.

In [ ]:
# Exercise 2.3: Configure plan mode

messages = []

async with ClaudeSDKClient(
    options=ClaudeAgentOptions(
        model="claude-sonnet-4-5",
        
        # TODO: Set permission_mode to "plan"
        permission_mode="",  # Fill this in
        
        allowed_tools=["Read", "WebSearch", "Bash"],
    )
) as agent:
    await agent.query(
        "Create a plan to analyze our sales data and generate a quarterly report. "
        "Don't execute - just outline the steps."
    )
    async for msg in agent.receive_response():
        print_activity(msg)
        messages.append(msg)

# Print the plan
if messages and hasattr(messages[-1], 'result'):
    print(f"\n📋 Generated Plan:\n{messages[-1].result[:1000]}...")

**Verification:** The agent should produce a plan without actually executing any tools (except possibly reading files to understand the context).

---

### Exercise 2.4: Complete Agent Function with All Features (20 points)

**Scenario:** Marcus is ready for the full implementation. You need to write a complete `send_query` function that integrates:
- Model configuration
- Tool permissions including Task for subagents
- Working directory
- Optional output style
- Optional permission mode
- Conversation continuation

**Your Task:** Complete the `send_query` function by filling in the missing parts of `ClaudeAgentOptions`.

**Hint:** Look at the `chief_of_staff_agent/agent.py` implementation for reference.

In [ ]:
# Exercise 2.4: Complete agent function implementation

import asyncio
from typing import Literal

async def send_ops_query(
    prompt: str,
    continue_conversation: bool = False,
    permission_mode: Literal["default", "plan", "acceptEdits"] = "default",
    output_style: str | None = None,
) -> tuple[str | None, list]:
    """
    Send a query to the Operations agent with enterprise features.
    
    Args:
        prompt: The query to send
        continue_conversation: Continue the previous conversation
        permission_mode: "default", "plan", or "acceptEdits"
        output_style: Optional output style (e.g., "executive", "technical")
    
    Returns:
        Tuple of (result, messages)
    """
    
    # Build the options dictionary
    options_dict = {
        # TODO: Set the model
        "model": "",  # Fill this in
        
        # TODO: Set allowed_tools to include Task (for subagents), Read, Write, Bash, WebSearch
        "allowed_tools": [],  # Fill this in
        
        # TODO: Set continue_conversation from the parameter
        "continue_conversation": None,  # Fill this in
        
        # TODO: Set permission_mode from the parameter
        "permission_mode": None,  # Fill this in
        
        # TODO: Set cwd to "chief_of_staff_agent"
        "cwd": "",  # Fill this in
        
        "system_prompt": "You are the Operations Chief of Staff for DataFlow Labs.",
    }
    
    # Add output style if specified
    if output_style:
        # TODO: Add settings with outputStyle to options_dict
        # Remember: settings takes a JSON string
        options_dict["settings"] = ""  # Fill this in using json.dumps()
    
    options = ClaudeAgentOptions(**options_dict)
    
    result = None
    messages = []
    
    async with ClaudeSDKClient(options=options) as agent:
        await agent.query(prompt=prompt)
        async for msg in agent.receive_response():
            messages.append(msg)
            print_activity(msg)
            
            if hasattr(msg, "result"):
                result = msg.result
    
    return result, messages


# Test your implementation
print("Testing your send_ops_query function...\n")

result, messages = await send_ops_query(
    "What's the current runway for TechStart Inc?",
    output_style="executive"
)

if result:
    print(f"\n✅ Function works! Response:\n{result[:500]}...")
else:
    print("\n❌ No result returned. Check your implementation.")

---

## Phase 2 Complete! 🎉

**Points Possible: 55**

You've demonstrated understanding of:
- ✅ Output style configuration via settings JSON
- ✅ Subagent markdown definition format
- ✅ Permission modes (default, plan, acceptEdits)
- ✅ Complete agent function with all enterprise features
- ✅ Working directory (cwd) configuration

Marcus is impressed with the enterprise-grade system. Now it's time for Week 3 - external integrations!

---

# Phase 3: External Integrations (Week 3)

## DataFlow Labs - DevOps Team Request

**From:** Jordan Lee, DevOps Lead  
**To:** You (AI/Agent Engineer)  
**Subject:** CI/CD Monitoring Agent

> *"We need an observability agent that can monitor our GitHub repositories, check CI/CD pipeline status, and help us respond to incidents. This requires connecting to external systems through MCP servers."*

This phase tests your understanding of **Notebook 02: Observability Agent** concepts.

---

### Exercise 3.1: MCP Server Configuration (15 points)

**Scenario:** Jordan needs you to configure an MCP server to connect to Git repositories. You need to understand the MCP server dictionary structure.

**Your Task:** Complete the Git MCP server configuration dictionary.

**Requirements:**
1. Server name key: `git`
2. Command: `uv`
3. Args: `["run", "python", "-m", "mcp_server_git", "--repository", <current_directory>]`

**Hint:** MCP server configs have `command`, `args`, and optionally `env` keys.

In [ ]:
# Exercise 3.1: Configure the Git MCP server

from typing import Any

# TODO: Complete the MCP server configuration
git_mcp_config: dict[str, Any] = {
    # TODO: Add the "git" server configuration
    # It should have "command" and "args" keys
    "": {  # Fill in the server name
        "command": "",  # Fill in the command (uv)
        "args": [],  # Fill in the args list
    }
}

# Validation
print("Your MCP server configuration:")
print(json.dumps(git_mcp_config, indent=2))

# Check the configuration
if "git" in git_mcp_config:
    config = git_mcp_config["git"]
    print("\n✅ Server name 'git' is correct")
    
    if config.get("command") == "uv":
        print("✅ Command is correct")
    else:
        print(f"❌ Command should be 'uv', got '{config.get('command')}'")
    
    args = config.get("args", [])
    if "mcp_server_git" in str(args) and "--repository" in args:
        print("✅ Args look correct")
    else:
        print("❌ Args should include 'mcp_server_git' module and '--repository' flag")
else:
    print("\n❌ Missing 'git' server in configuration")

---

### Exercise 3.2: MCP Tool Permissions (10 points)

**Scenario:** Now you need to configure the agent to use the MCP tools. MCP tools follow a specific naming convention.

**Your Task:** Complete the `allowed_tools` list with the correct MCP tool permission format.

**Requirements:**
1. Allow all tools from the `git` MCP server
2. Also allow `Read` and `WebSearch`

**Hint:** MCP tool permissions use the format `mcp__<serverName>` for all tools from a server, or `mcp__<serverName>__<toolName>` for specific tools.

In [ ]:
# Exercise 3.2: Configure MCP tool permissions

# TODO: Complete the allowed_tools list
allowed_tools = [
    # TODO: Add permission for all git MCP tools (format: mcp__serverName)
    "",  # Fill this in
    
    # Also include standard tools
    "Read",
    "WebSearch",
]

# Validation
print("Your allowed_tools configuration:")
print(allowed_tools)

# Check the configuration
if any("mcp__git" in tool for tool in allowed_tools):
    print("\n✅ MCP git tools are correctly formatted")
else:
    print("\n❌ Missing MCP git tools. Format should be 'mcp__git'")

if "Read" in allowed_tools and "WebSearch" in allowed_tools:
    print("✅ Standard tools included")
else:
    print("❌ Missing Read or WebSearch")

---

### Exercise 3.3: GitHub MCP with Docker (15 points)

**Scenario:** For production, you need to configure the GitHub MCP server which runs in Docker. This requires environment variables for authentication.

**Your Task:** Complete the GitHub MCP server configuration.

**Requirements:**
1. Server name: `github`
2. Command: `docker`
3. Args: Docker run command with the GitHub MCP server image
4. Environment: Pass the `GITHUB_PERSONAL_ACCESS_TOKEN`

**Hint:** Look at the observability_agent's `GITHUB_MCP_SERVER` configuration.

In [ ]:
# Exercise 3.3: Configure GitHub MCP server with Docker

# TODO: Complete the GitHub MCP server configuration
github_mcp_config: dict[str, Any] = {
    "": {  # Fill in the server name (github)
        "command": "",  # Fill in the command (docker)
        "args": [
            # TODO: Fill in the docker run arguments
            # Should be: run, -i, --rm, -e, GITHUB_PERSONAL_ACCESS_TOKEN, <image>
            "",  # run
            "",  # -i (interactive)
            "",  # --rm (remove container after exit)
            "",  # -e (environment variable)
            "",  # GITHUB_PERSONAL_ACCESS_TOKEN
            "",  # ghcr.io/github/github-mcp-server (the image)
        ],
        "env": {
            # TODO: Map the environment variable to the actual token
            "": os.environ.get(""),  # Fill in both keys
        },
    }
}

# Validation
print("Your GitHub MCP server configuration:")
print(json.dumps({k: {**v, 'env': '***'} for k, v in github_mcp_config.items()}, indent=2))

# Check the configuration
if "github" in github_mcp_config:
    config = github_mcp_config["github"]
    print("\n✅ Server name 'github' is correct")
    
    if config.get("command") == "docker":
        print("✅ Command is correct (docker)")
    else:
        print(f"❌ Command should be 'docker', got '{config.get('command')}'")
    
    args = config.get("args", [])
    if "run" in args and "-i" in args and "--rm" in args:
        print("✅ Docker args include run, -i, --rm")
    else:
        print("❌ Docker args should include 'run', '-i', '--rm'")
        
    if "ghcr.io/github/github-mcp-server" in args:
        print("✅ Correct Docker image specified")
    else:
        print("❌ Missing Docker image: ghcr.io/github/github-mcp-server")
    
    if "env" in config and "GITHUB_PERSONAL_ACCESS_TOKEN" in config["env"]:
        print("✅ Environment variable configured correctly")
    else:
        print("❌ Missing GITHUB_PERSONAL_ACCESS_TOKEN in env")
else:
    print("\n❌ Missing 'github' server in configuration")

---

### Exercise 3.4: Complete Observability Agent Options (10 points)

**Scenario:** Now put it all together! Create the complete `ClaudeAgentOptions` for an observability agent that uses MCP servers.

**Your Task:** Complete all the options for the observability agent.

**Note:** This exercise doesn't run the actual agent (which would require Docker), but validates your configuration knowledge.

In [ ]:
# Exercise 3.4: Complete observability agent configuration

# Define the MCP servers (using your configurations from above)
mcp_servers = {
    "github": {
        "command": "docker",
        "args": [
            "run", "-i", "--rm", "-e", 
            "GITHUB_PERSONAL_ACCESS_TOKEN",
            "ghcr.io/github/github-mcp-server"
        ],
        "env": {"GITHUB_PERSONAL_ACCESS_TOKEN": os.environ.get("GITHUB_TOKEN")},
    }
}

# TODO: Complete the ClaudeAgentOptions configuration
observability_options = ClaudeAgentOptions(
    # TODO: Set the model
    model="",  # Fill this in
    
    # TODO: Set mcp_servers
    mcp_servers=None,  # Fill this in with the mcp_servers variable
    
    # TODO: Set allowed_tools for GitHub MCP and standard tools
    allowed_tools=[],  # Fill this in
    
    # TODO: Set permission_mode to acceptEdits for production safety
    permission_mode="",  # Fill this in
    
    # TODO: Add a system prompt for an observability agent
    system_prompt="",  # Fill this in
)

# Validation
print("Your observability agent configuration:")
print(f"  Model: {observability_options.model}")
print(f"  MCP Servers: {list(observability_options.mcp_servers.keys()) if observability_options.mcp_servers else 'None'}")
print(f"  Allowed Tools: {observability_options.allowed_tools}")
print(f"  Permission Mode: {observability_options.permission_mode}")
print(f"  System Prompt: {observability_options.system_prompt[:50]}..." if observability_options.system_prompt else "None")

# Check configuration
errors = []
if not observability_options.model:
    errors.append("Model not set")
if not observability_options.mcp_servers:
    errors.append("MCP servers not configured")
if not observability_options.allowed_tools or not any("mcp__github" in str(t) for t in observability_options.allowed_tools):
    errors.append("MCP tools not in allowed_tools")
if observability_options.permission_mode != "acceptEdits":
    errors.append("Permission mode should be 'acceptEdits'")
if not observability_options.system_prompt:
    errors.append("System prompt not set")

if errors:
    print(f"\n❌ Issues found: {', '.join(errors)}")
else:
    print("\n✅ Configuration looks complete!")

---

## Phase 3 Complete! 🎉

**Points Possible: 50**

You've demonstrated understanding of:
- ✅ MCP server configuration structure
- ✅ MCP tool permission naming convention
- ✅ Docker-based MCP servers with environment variables
- ✅ Complete observability agent configuration
- ✅ Permission modes for production safety

Jordan and the DevOps team now have the foundation for their observability system!

---

# 📚 Feature Reference Guide

Before tackling the Final Challenge, here's a comprehensive reference of all the features you've learned. Use this as your glossary when designing agents and exploring on your own.

---

## Core SDK Patterns (Notebook 00)

These are the foundational building blocks for any agent:

| Feature | Parameter/Pattern | Description | When to Use |
|---------|-------------------|-------------|-------------|
| **Stateless queries** | `query()` | One-off interactions with no memory | Independent research tasks, parallel processing |
| **Stateful conversations** | `ClaudeSDKClient` | Multi-turn context maintained | Iterative refinement, complex analysis |
| **Tool permissions** | `allowed_tools=["WebSearch", "Read"]` | List of tools the agent can use | Control what actions the agent can take |
| **System prompts** | `system_prompt="You are a..."` | Specialize agent behavior and expertise | Domain-specific agents |
| **Activity handlers** | `def handler(msg): ...` | Callbacks for real-time feedback | Logging, progress indicators, custom UI |
| **Conversation continuation** | `continue_conversation=True` | Resume previous conversation state | Follow-up questions, iterative work |

### Code Pattern: Basic Agent Loop
```python
async with ClaudeSDKClient(options=ClaudeAgentOptions(
    model="claude-sonnet-4-5",
    system_prompt="Your specialization here",
    allowed_tools=["WebSearch", "Read"],
)) as agent:
    await agent.query("Your prompt")
    async for msg in agent.receive_response():
        print_activity(msg)
```

---

## Enterprise Features (Notebook 01)

These features enable production-grade, multi-user agent systems:

| Feature | Parameter/Pattern | Description | When to Use |
|---------|-------------------|-------------|-------------|
| **Memory (CLAUDE.md)** | `cwd="path/to/dir"` | Persistent context loaded from CLAUDE.md file | Company knowledge, project context |
| **Working directory** | `cwd="chief_of_staff_agent"` | Sets agent's file system context | Access to scripts, data files, configs |
| **Bash tool** | `allowed_tools=["Bash"]` | Run Python scripts and shell commands | Financial models, data processing |
| **Output styles** | `settings='{"outputStyle": "executive"}'` | Different communication formats | Executive vs. technical audiences |
| **Permission modes** | `permission_mode="plan"` | Control execution behavior | `plan` (think only), `acceptEdits` (auto-approve), `default` (execute) |
| **Slash commands** | `/budget-impact args` | User-friendly prompt shortcuts | Recurring queries, standardization |
| **Hooks** | `.claude/hooks/` + `settings.local.json` | Automated pre/post tool actions | Compliance, audit trails, guardrails |
| **Subagents** | `allowed_tools=["Task"]` | Delegate to specialized agents | Domain experts, parallel work |

### Configuration Files Structure
```
your_agent/
├── CLAUDE.md                    # Agent memory/context
├── .claude/
│   ├── agents/
│   │   └── specialist.md        # Subagent definitions
│   ├── commands/
│   │   └── my-command.md        # Slash command templates
│   ├── output-styles/
│   │   └── executive.md         # Output format rules
│   ├── hooks/
│   │   └── audit-logger.py      # Hook scripts
│   └── settings.local.json      # Hook configuration
└── scripts/
    └── analysis.py              # Python utilities
```

### Subagent Definition Format
```markdown
---
name: specialist-name
description: What this agent does (shown to parent agent)
tools: Read, Bash, WebSearch
---

You are a specialist in...

## Your Responsibilities
1. First responsibility
2. Second responsibility
```

### Output Style Format
```markdown
---
name: executive
description: Concise, KPI-focused communication
---

## Communication Principles
- Lead with recommendations
- Use bullet points
- Specific metrics and timeframes
```

---

## External Integrations (Notebook 02)

These features connect your agent to external systems via Model Context Protocol:

| Feature | Parameter/Pattern | Description | When to Use |
|---------|-------------------|-------------|-------------|
| **MCP servers** | `mcp_servers={"git": {...}}` | Connect to external systems | GitHub, databases, APIs |
| **MCP tool permissions** | `allowed_tools=["mcp__github"]` | Allow tools from MCP server | `mcp__serverName` or `mcp__serverName__toolName` |
| **Docker MCP** | `"command": "docker"` | Containerized server | Production isolation, security |
| **Environment variables** | `"env": {"TOKEN": os.environ.get("KEY")}` | Pass secrets to MCP servers | Authentication, API keys |

### MCP Server Configuration Pattern
```python
mcp_servers = {
    "server_name": {
        "command": "uv",  # or "docker"
        "args": ["run", "python", "-m", "module_name", "--flag", "value"],
        "env": {  # Optional
            "API_KEY": os.environ.get("MY_API_KEY")
        }
    }
}
```

### MCP Tool Permission Formats
```python
allowed_tools = [
    "mcp__github",                    # All tools from github server
    "mcp__github__search_repositories", # Specific tool only
    "mcp__git__git_log",              # Specific git tool
    "Read",                           # Standard tools still work
]
```

---

## Quick Reference: ClaudeAgentOptions Parameters

| Parameter | Type | Description |
|-----------|------|-------------|
| `model` | `str` | Model to use (e.g., `"claude-sonnet-4-5"`) |
| `system_prompt` | `str` | Agent specialization instructions |
| `allowed_tools` | `list[str]` | Tools the agent can use freely |
| `cwd` | `str` | Working directory path |
| `settings` | `str` | JSON string with settings like `outputStyle` |
| `permission_mode` | `str` | `"default"`, `"plan"`, or `"acceptEdits"` |
| `continue_conversation` | `bool` | Resume previous conversation |
| `mcp_servers` | `dict` | MCP server configurations |

---

## Example Feature Combinations

Here are some ideas for combining features effectively:

### Marketing Agent
- **Output styles**: Different formats for CMO vs. content team
- **Subagents**: Content writer, analytics specialist, SEO expert
- **Hooks**: Brand compliance checking before publishing

### HR Agent
- **Memory (CLAUDE.md)**: Company policies, org structure, benefits info
- **Bash for scripts**: Compensation calculator, PTO tracker
- **Permission modes**: Plan mode for sensitive decisions like terminations

### Customer Success Agent
- **MCP server**: Zendesk/Intercom integration for ticket data
- **System prompt**: Empathetic, solution-focused tone
- **Output styles**: Customer-facing vs. internal escalation reports

### Finance Agent
- **Subagents**: Tax specialist, auditor, forecasting analyst
- **Hooks**: Audit trail for all financial calculations
- **Bash for scripts**: Financial models, scenario analysis

### DevOps Agent
- **MCP servers**: GitHub + PagerDuty + AWS
- **Permission mode**: `acceptEdits` for automated fixes
- **System prompt**: Focus on root cause analysis and prevention

---

## Terminology Glossary

| Term | Definition |
|------|------------|
| **Agent** | An AI system that can use tools autonomously to accomplish tasks |
| **Stateless** | No memory between interactions; each query is independent |
| **Stateful** | Maintains conversation context across multiple queries |
| **Tool** | A capability the agent can invoke (WebSearch, Read, Bash, etc.) |
| **Subagent** | A specialized agent invoked by a parent agent via the Task tool |
| **MCP** | Model Context Protocol - standard for AI-tool integrations |
| **Hook** | Automated script that runs before/after specific tool calls |
| **Output style** | Predefined communication format for different audiences |
| **Permission mode** | Controls whether agent executes actions or just plans |
| **Working directory (cwd)** | The file system context where the agent operates |
| **Frontmatter** | YAML metadata at the top of markdown files (between `---` markers) |
| **Activity handler** | Callback function that processes agent messages in real-time |

---

Now you're ready for the Final Challenge! Choose features from the tables above to design your custom agent.

---

# Final Challenge: Design Your Own Agent (Bonus)

## DataFlow Labs - CEO Request

**From:** Alex Rivera, CEO  
**To:** You (AI/Agent Engineer)  
**Subject:** Custom Agent for Your Department

> *"You've proven yourself capable of building sophisticated agent systems. Now I want you to design an agent for a department of your choice. It should combine at least three features you've learned. Write out the configuration and explain your design decisions."*

**Requirements:**
1. Choose a department (Marketing, HR, Finance, Customer Success, etc.)
2. Include at least 3 features from what you've learned
3. Write a complete configuration
4. Explain your design choices

This is an open-ended exercise to demonstrate your comprehensive understanding.

In [ ]:
# Final Challenge: Design your own agent

# Department: _______________ (fill in your choice)
# Features included:
# 1. _______________
# 2. _______________  
# 3. _______________

# Your agent configuration:
my_agent_options = ClaudeAgentOptions(
    model="claude-sonnet-4-5",
    
    # Add your configuration here
    # ...
)

# Design explanation:
"""
Write 3-5 sentences explaining:
1. Why you chose this department
2. Why you selected these specific features
3. How this agent would help the team
"""

print("Your custom agent design is ready for review!")

---

# Conclusion

## Assessment Complete! 🎓

You've completed the Claude Agent SDK Learning Path Assessment. Here's what you've demonstrated:

### Phase 1: Foundations (35 points)
- Stateless queries with `query()`
- Stateful conversations with `ClaudeSDKClient`
- Tool permissions and system prompts
- Activity handler patterns

### Phase 2: Enterprise Features (55 points)
- Output style configuration
- Subagent definition format
- Permission modes
- Complete agent function implementation

### Phase 3: External Integrations (50 points)
- MCP server configuration
- MCP tool permissions
- Docker-based MCP servers
- Complete observability agent

### Total Possible: 140+ points

---

## Next Steps

Now that you've mastered the fundamentals, here are ways to deepen your expertise:

1. **Build a Real Agent**: Apply these patterns to a real use case in your work
2. **Explore Custom MCP Servers**: Build your own MCP server for a specific integration
3. **Production Deployment**: Learn about scaling, monitoring, and security for production agents
4. **Multi-Agent Orchestration**: Design complex workflows with multiple specialized agents

---

## Resources

- [Claude Agent SDK Documentation](https://docs.anthropic.com/en/docs/agents-and-tools/claude-agent-sdk)
- [Model Context Protocol](https://modelcontextprotocol.io/)
- [Anthropic Cookbook](https://github.com/anthropics/anthropic-cookbook)
- [Building Effective Agents](https://www.anthropic.com/engineering/building-effective-agents)

---

*Congratulations on completing the learning path! You're now ready to build production-grade agent systems with the Claude Agent SDK.*